In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [4]:
pbar = tqdm()

0it [00:00, ?it/s]

#### Run all operators

In [20]:
analysis_date = dt.date(2022, 5, 29) ##wednesday, new tables

In [21]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [22]:
## TODO track ran on date!
# ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
#                  for path in fs_list
#                  if path.split('rt_trips/')[1]]

In [23]:
ran_operators = []

In [24]:
not_ran_operators = []
# for agency in air_joined.calitp_itp_id.unique():
for agency in [87]:
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    ## https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.export_views_gcs()
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 87...
getting trips...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


complete for agency: 87


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [25]:
import yaml

In [26]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    # >> filter(_.itp_id == itp_id)
    >> collect()
)

In [10]:
airtable_organizations = airtable_organizations.dropna(subset=['itp_id']) >> mutate(itp_id = _.itp_id.astype('int64'))

In [11]:
airtable_organizations = airtable_organizations >> filter(_.itp_id.isin(ran_operators)) >> arrange(_.caltrans_district)

In [12]:
airtable_organizations.head(3)

,itp_id,name,caltrans_district,website,ntp_id,drmt_organization_name
663,135,Humboldt Transit Authority,01 - Eureka,None,['reccKueWkvBCZoLQs'],['recu9h9nAElLpYoYO']
701,159,Lake Transit Authority,01 - Eureka,None,['recF9AnmCtEyxxDTY'],['reck7ddrpXSeA0mH0']
572,259,Redding Area Bus Authority,02 - Redding,None,['recoXZjKEOPldH8QJ'],None


In [13]:
with open('../portfolio/analyses.yml') as analyses:
    analyses_data = yaml.load(analyses, yaml.Loader)

In [14]:
exclude_ids = [312]

In [15]:
chapters_list = []
for district in airtable_organizations.caltrans_district.unique():
    chapter_dict = {}
    filtered = (airtable_organizations
                >> filter(_.caltrans_district == district)
                >> distinct(_.itp_id, _keep_all=True)
                >> filter(-_.itp_id.isin(exclude_ids))
               )
    chapter_dict['caption'] = f'District {district}'
    chapter_dict['params'] = {'district': district}
    chapter_dict['sections'] = [{'itp_id': itp_id} for itp_id in filtered.itp_id.to_list()]
    chapters_list += [chapter_dict]

In [16]:
parts_list = [{'chapters': chapters_list}]

In [17]:
analyses_data['sites'][1]['parts'] = parts_list

In [18]:
output = yaml.dump(analyses_data)

In [19]:
with open('../portfolio/analyses.yml', 'w') as analyses:
    analyses.write(output)

### Ongoing issues
* 358 Union City too many trips without shape?

#### Parameters
* 159 1+ maps don't render
* 167 map_from_metrics fails